In [1]:
import collections
import logging
import os
from typing import Optional, Tuple
#from tqdm.auto import tqdm

# misc
import re
import time
import ast
import warnings
import math
import copy
import matplotlib.pyplot as plt
# from xgboost import plot_importance
# import seaborn as sns

# data
import pandas as pd
import numpy as np
import csv
from torch.utils.data import random_split
import datasets

# ML
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn import metrics

#import scikitplot as skplt
# import xgboost
# import tensorflow as tf
import torch

from transformers import DefaultDataCollator, EvalPrediction
from transformers import AutoTokenizer, AutoModelForSequenceClassification, XLMRobertaForSequenceClassification
from transformers import DistilBertConfig, DistilBertModel, BertTokenizerFast
from transformers import AutoModel
from datasets import load_dataset, DatasetDict, Dataset
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, BertForQuestionAnswering, AutoModelForQuestionAnswering, DistilBertForSequenceClassification, DistilBertModel, CamembertForSequenceClassification, RobertaForSequenceClassification
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm
import json
from transformers import TrainerCallback, EarlyStoppingCallback
from sklearn.metrics import accuracy_score
from collections import Counter
from torch.utils.data import DataLoader
import evaluate
import optuna

c:\Users\Alex\miniconda3\envs\pytorch-modified\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
squad_raw = load_dataset("squad_v2", split="train[:100]+validation[:20]")
train_testvalid = squad_raw.train_test_split(test_size=0.4, shuffle=True)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5, shuffle=True)

squad_raw = datasets.DatasetDict({
                                'train': train_testvalid['train'],
                                'validation': test_valid['train'],
                                'test': test_valid['test']
                                })

In [3]:
def find_positions(dataset):
    dataset_answers = dataset['answers']
    dataset_contexts = dataset['context']
    updated_answers = []

    for i, (answer, text) in enumerate(zip(dataset_answers, dataset_contexts)):
        if len(answer['answer_start']) == 0:
            answer['answer_end'] = []
            updated_answers.append(answer)
            continue
        
        real_answer = answer['text'][0]
        real_answer_length = len(real_answer)
        start_idx = answer['answer_start'][0]
        end_idx = start_idx + real_answer_length

        if text[start_idx:end_idx] == real_answer:
            answer['answer_end'] = [end_idx]
        elif text[start_idx-1:end_idx-1] == real_answer:
            answer['answer_start'] = [start_idx-1]
            answer['answer_end'] = [end_idx-1]
        elif text[start_idx-2:end_idx-2] == real_answer:
            answer['answer_start'] = [start_idx-2]
            answer['answer_end'] = [end_idx-2]
        updated_answers.append(answer)

    for answer in updated_answers:
        if 'answer_end' not in answer:
            answer['answer_end'] = []
            
    updated_dataset = {
        'id': dataset['id'],
        'title': dataset['title'],
        'context': dataset['context'],
        'question': dataset['question'],
        'answers': updated_answers
    }

    return updated_dataset

In [4]:
squad_raw_train = find_positions(squad_raw["train"])
squad_raw_val = find_positions(squad_raw["validation"])
squad_raw_test = find_positions(squad_raw["test"])

In [5]:
model_name = "bert-base-uncased"

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

train_texts = squad_raw_train["context"]
train_queries = squad_raw_train["question"]
val_texts = squad_raw_val["context"]
val_queries = squad_raw_val["question"]

c:\Users\Alex\miniconda3\envs\pytorch-modified\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
train_encodings = tokenizer(train_texts, train_queries, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, val_queries, truncation=True, padding=True)

In [8]:
def add_token_positions(encodings, answers):
  start_positions = []
  end_positions = []
  count = 0
  
  for i in range(len(answers)):
    temp_start = None
    temp_end = None
    
    if len(answers[i]['answer_start']) != 0:
      temp_start = answers[i]['answer_start'][0]
      temp_end = answers[i]['answer_end'][0]

    start_positions.append(encodings.char_to_token(i, temp_start))
    end_positions.append(encodings.char_to_token(i, temp_end))
    
    if start_positions[-1] is None:
      start_positions[-1] = tokenizer.model_max_length
      
    if end_positions[-1] is None:
      if end_positions[-1] is not None:
        end_positions[-1] = encodings.char_to_token(i, temp_end + 1)
      if end_positions[-1] is None:
        count += 1
        end_positions[-1] = tokenizer.model_max_length

  print(count)
  encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, squad_raw_train["answers"])
add_token_positions(val_encodings, squad_raw_val["answers"])

34
19


In [52]:
train_examples = [train_encodings]

# Convert to list
train_list = []
for sample in train_examples:
  train_list.extend(sample.to_dict().values()) 

def batch_data(examples, batch_size):
  batched = {}
  
  for key in examples[0]:
    batched[key] = [sample[key] for sample in examples[:batch_size]]

  return batched

train_batches = []

for i in range(0, len(train_list), batch_size):
  batch = train_list[i:i+batch_size]
  train_batches.append(batch_data(batch, batch_size))

AttributeError: 

In [50]:
batch_size = 32

class SquadDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)

def batch_dataset(dataset):
  batched = {}
  for key in dataset[0]:
    batched[key] = [ sample[key] for sample in dataset]
  return batched

train_dataset = SquadDataset(train_encodings)
val_dataset = SquadDataset(val_encodings)

batched_train_dataset = train_dataset.map(batch_dataset, batched=True, batch_size=batch_size)
batched_val_dataset = val_dataset.map(batch_dataset, batched=True, batch_size=batch_size)

AttributeError: 'SquadDataset' object has no attribute 'map'

In [ ]:
def batch_dataset(examples):
  batched_examples = {"input_ids": torch.tensor([x["input_ids"] for x in examples])}

  return batched_examples

In [42]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [15]:
torch.cuda.empty_cache()

In [46]:
data_collator = DefaultDataCollator()
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

training_args = TrainingArguments(
    output_dir="squadv2",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train_dataloader = train_loader
trainer.eval_dataloader = val_loader
trainer.train()

2024-05-24 21:34:58,286 - DEBUG - https://huggingface.co:443 "HEAD /bert-base-uncased/resolve/main/config.json HTTP/1.1" 200 0
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/3 [01:18<?, ?it/s]


ValueError: Trainer: training requires a train_dataset.